### Import all necessary libraries for analysis


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import os

### Reading the data into pandas dataframe

In [2]:
Amazon_Prime = pd.read_csv('Amazon_Prime_Users.csv')
print("Rows, cols:", Amazon_Prime.shape)
display(Amazon_Prime.head(5))
Amazon_Prime.info()

Rows, cols: (2530, 19)


,User ID,Name,Email Address,Username,Date of Birth,Gender,Location,Membership Start Date,Membership End Date,Subscription Plan,Payment Information,Renewal Status,Usage Frequency,Purchase History,Favorite Genres,Devices Used,Engagement Metrics,Feedback/Ratings,Customer Support Interactions
0,1,Ronald Murphy,williamholland@example.com,williamholland,3/6/1953,Male,Rebeccachester,15/1/2024,14/1/2025,Annual,Mastercard,Manual,Regular,Electronics,Documentary,Smart TV,Medium,3.6,3.0
1,2,Scott Allen,scott22@example.org,scott22,8/7/1978,Male,Mcphersonview,7/1/2024,6/1/2025,Monthly,Visa,Manual,Regular,Electronics,Horror,Smartphone,Medium,3.8,7.0
2,3,Jonathan Parrish,brooke16@example.org,brooke16,6/12/1994,Female,Youngfort,13/4/2024,13/4/2025,Monthly,Mastercard,Manual,Regular,Books,Comedy,Smart TV,Low,3.3,8.0
3,4,Megan Williams,elizabeth31@example.net,elizabeth31,22/12/1964,Female,Feliciashire,24/1/2024,23/1/2025,Monthly,Amex,Auto-renew,Regular,Electronics,Documentary,Smart TV,High,3.3,7.0
4,5,Kathryn Brown,pattersonalexandra@example.org,pattersonalexandra,4/6/1961,Male,Port Deborah,14/2/2024,13/2/2025,Annual,Visa,Auto-renew,Frequent,Clothing,Drama,Smart TV,Low,4.3,1.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2530 entries, 0 to 2529
Data columns (total 19 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   User ID                        2530 non-null   int64  
 1   Name                           2530 non-null   object 
 2   Email Address                  2530 non-null   object 
 3   Username                       2530 non-null   object 
 4   Date of Birth                  2530 non-null   object 
 5   Gender                         2530 non-null   object 
 6   Location                       2530 non-null   object 
 7   Membership Start Date          2530 non-null   object 
 8   Membership End Date            2530 non-null   object 
 9   Subscription Plan              2530 non-null   object 
 10  Payment Information            2530 non-null   object 
 11  Renewal Status                 2530 non-null   object 
 12  Usage Frequency                2530 non-null   o

In [3]:
Amazon_Prime = Amazon_Prime.dropna(subset=['Feedback/Ratings', 'Customer Support Interactions'])

### Question 1 :

Does the younger user age group (18–30) associate with higher likelihood of rating Prime Video content as “excellent” when they also have high streaming frequency?

In [4]:
Amazon_Prime['Date of Birth'] = pd.to_datetime(Amazon_Prime['Date of Birth'], errors='coerce')
Amazon_Prime['age'] = (pd.Timestamp.today() - Amazon_Prime['Date of Birth']).dt.days // 365

Amazon_Prime['age_group'] = pd.cut(Amazon_Prime['age'], 
                         bins=[0,18,30,40,60,200], 
                         labels=['<18','18-30','31-40','41-60','60+'])

Amazon_Prime['age_group'].value_counts()


age_group
60+      440
41-60    259
31-40    144
18-30    129
<18        0
Name: count, dtype: int64

In [5]:
usage_mapping = {
    'Occasional': 'Low',
    'Regular': 'Medium',
    'Frequent': 'High'
}

Amazon_Prime['usage_bin'] = Amazon_Prime['Usage Frequency'].map(usage_mapping)

